Deep Learning
=============

`eval()` vs `session.run()`

In [15]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

In [16]:
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are attached to the graph.
  # az egész adathalmaz 1 db változó lesz! :)
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    
    tf.truncated_normal([image_size * image_size, num_labels])) # így 82.5% és 82.7% lett
    
    #tf.zeros([image_size * image_size, num_labels])) # így 88.6% és 88.6% lett!!! XXX XXX XXX (ez -35%!)

  biases = tf.Variable(tf.zeros([num_labels]))
  # az egész W az 1 db változó és a b is :)
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.

  # szinte pont, ahogy volt a diákon (csak már az első lépésben a teljes adathalmazon dolgozunk):
  # XW+b for every X_i in input set
  # (azaz itt a biases-nél van egy broadcasting, minden sorhoz hozzáadja!):
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # 1/N sum_i D(S(fenti),L) ::
  # XXX a reduce_mean() honnan is tudja, hogy mely változók szerint? XXX
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  # 0.5 -- ez hát a learning rate! :) + a minimize-ben adom meg, hogy mi szerint
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  # -- értsd: minden nyilván az adott változó épp aktuális értékéből számolva!
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    
    session.run(valid_prediction)
    valid_prediction.eval()
    
    session.run(optimizer)
    optimizer.eval()

AttributeError: 'Operation' object has no attribute 'eval'

**Eredmény:** `Tensor`-ra megy mindkettő, `Operation`-re nincs `eval()` csak a `session.run()`!